In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os


In [2]:
!pip install pandas openpyxl

In [3]:
files = os.listdir(r"/Users/ikawahyuni/Desktop/The Key/file XLSX")
files

['.DS_Store',
 'lister_data_media_output.xlsx',
 'KI_instagram_data_media_output.xlsx',
 'EF_data_media_output.xlsx',
 'BC_data_media_output.xlsx',
 'the_key_data_media_output.xlsx',
 'WSE_data_media_output.xlsx',
 'titik_nol_data_media_output.xlsx']

In [4]:
# we made a function so if we need to read any data we just call this function

path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"
def read_data(path, *filenames):
    data = []
    for file in filenames:
        full_path = os.path.join(path, file)
        df = pd.read_excel(full_path)
        key = file.replace(".xlsx", "")
        data[key] = df

    # kalau cuma 1 file → return DataFrame langsung
    if len(data) == 1:
        return list(data.values())[0]  # return only df

    return data






path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"
data_dict = read_data(
    path,
    'KI_instagram_data_media_output.xlsx',
    'EF_data_media_output.xlsx',
    'BC_data_media_output.xlsx',
    'the_key_data_media_output.xlsx',
    'WSE_data_media_output.xlsx',
    'Lister.xlsx',
    'titik_nol_data_media_output.xlsx'
)

# Content Performance Analysis
## 1. Media Product Type

In [5]:
import pandas as pd
import plotly.express as px 


# ==========================================
# 1. Daftar file + nama account
# ==========================================
files = {
    "BritishCouncil": "/Users/ikawahyuni/Desktop/The Key/file XLSX/BC_data_media_output.xlsx",
    "WallStreetEnglish": "/Users/ikawahyuni/Desktop/The Key/file XLSX/WSE_data_media_output.xlsx",
    "EF": "/Users/ikawahyuni/Desktop/The Key/file XLSX/EF_data_media_output.xlsx",
    "Lister": "/Users/ikawahyuni/Desktop/The Key/file XLSX/lister_data_media_output.xlsx",
    "Titik_Nol": "/Users/ikawahyuni/Desktop/The Key/file XLSX/titik_nol_data_media_output.xlsx",
    "KampungInggrisLC": "/Users/ikawahyuni/Desktop/The Key/file XLSX/KI_instagram_data_media_output.xlsx",
    "The_Key": "/Users/ikawahyuni/Desktop/The Key/file XLSX/the_key_data_media_output.xlsx"
   
}

# ==========================================
# 2. Loop → baca semua file + tambahkan kolom account
# ==========================================
# Setelah loop selesai, nilai path bukan lagi folder.
df_list = []

for account, file_path in files.items():
    df = pd.read_excel(file_path)
    df["account"] = account  # beri label account
    df_list.append(df)

# ==========================================
# 3. Concat semua tabel
# ==========================================
#Setelah concat terdapat perbedaan struktur kolom, jd kolomnya ada yang tidak sama
# menyebabkan NaN tampil di df_all, meskipun dataframe aslinya terlihat tidak bermasalah.
df_all = pd.concat(df_list, ignore_index=True)
df_all["hour"] = pd.to_datetime(df_all["timestamp"]).dt.hour
df_all["date"] = pd.to_datetime(df_all["timestamp"]).dt.date
df_all.columns = df_all.columns.str.lower()





# ==========================================
# 4. Hitung jumlah FEED / REELS per account
# ==========================================
media_dist = df_all.groupby(["account", "media_product_type"]).size().reset_index(name="count")

print(media_dist)

# ==========================================
# 5. Visualisasi bar chart per account (mirip gambar kamu)
# ==========================================
fig = px.bar(media_dist, x='account', y='count', color='media_product_type', title='Media Product Type Distribution per account', barmode='group')
fig.show()


              account media_product_type  count
0      BritishCouncil               FEED     43
1      BritishCouncil              REELS      7
2                  EF               FEED     19
3                  EF              REELS     31
4    KampungInggrisLC               FEED     16
5    KampungInggrisLC              REELS     34
6              Lister               FEED     26
7              Lister              REELS     24
8             The_Key               FEED      2
9             The_Key              REELS     48
10          Titik_Nol               FEED     34
11          Titik_Nol              REELS     16
12  WallStreetEnglish               FEED     17
13  WallStreetEnglish              REELS     33


In [6]:
# try:
#     EF_data_media
# except NameError:
#     print("Membaca file dari Excel...")
#     folder_path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"
#     EF_data_media = pd.read_excel(os.path.join(folder_path, 'EF_data_media_output.xlsx'))


In [7]:
df_all['media_product_type'].unique().tolist()

['FEED', 'REELS']

In [8]:
#EF_data_media.columns = EF_data_media.columns.str.lower()
# EF_data_media['date'] = pd.to_datetime(EF_data_media['date'], errors='coerce')
# lister_data_media['date'] = pd.to_datetime(lister_data_media['date'], errors='coerce')
# #coercce digunakan untuk mengubah nilai yang tidak dapat dikonversi menjadi NaT (Not a Time)


## 2. 10 Top Performing Post


In [9]:
#1
# FIX: gunakan nama variabel folder_path untuk folder utama
folder_path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"

def read_data(folder_path, filename):
    full_path = os.path.join(folder_path, filename)
    return pd.read_excel(full_path)

BC_data_media = read_data(folder_path, 'BC_data_media_output.xlsx')


# ============================
# 2. Potong caption 50 karakter
# ============================
BC_data_media["caption_short"] = BC_data_media["caption"].astype(str).str.slice(0, 50)

# ============================
# 3. Hitung Engagement Rate
# Rumus: (likes + comments) / impressions * 100
# Jika impressions tidak ada → pakai reach
# ============================
# Hitung Engagement Rate berdasarkan impressions
BC_data_media["engagement_score"] = BC_data_media["like_count"] + BC_data_media["comments_count"]

max_eng = BC_data_media["engagement_score"].max()

BC_data_media["engagement_rate_percent"] = (BC_data_media["engagement_score"] / max_eng) * 100
BC_data_media["engagement_rate_percent"] = BC_data_media["engagement_rate_percent"].round(2)
BC_data_media["engagement_rate_percent"].head()

# 4. AMBIL 10 DENGAN ER TERTINGGI
top10 = BC_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)

fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - BC", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()


In [10]:
# EF _istagram 
folder_path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"

def read_data(folder_path, filename):
    full_path = os.path.join(folder_path, filename)
    return pd.read_excel(full_path)

EF_data_media = read_data(folder_path, 'EF_data_media_output.xlsx')

# ============================
# 2. Potong caption 50 karakter
# ============================
EF_data_media["caption_short"] = EF_data_media["caption"].astype(str).str.slice(0, 50)
# ============================
EF_data_media["caption_short"] = EF_data_media["caption"].astype(str).str.slice(0, 50)

# ============================
# 3. Hitung Engagement Rate
# Rumus: (likes + comments) / impressions * 100
# Jika impressions tidak ada → pakai reach
# ============================
# Hitung Engagement Rate berdasarkan impressions
EF_data_media["engagement_score"] = EF_data_media["like_count"] + EF_data_media["comments_count"]

max_eng = EF_data_media["engagement_score"].max()

EF_data_media["engagement_rate_percent"] = (EF_data_media["engagement_score"] / max_eng) * 100
EF_data_media["engagement_rate_percent"] = EF_data_media["engagement_rate_percent"].round(2)
EF_data_media["engagement_rate_percent"].head()

top10 = EF_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)

fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - EF", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()







In [11]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# WSE_Instagram
folder_path = r"/Users/ikawahyuni/Desktop/The Key/file XLSX"

def read_data(folder_path, filename):
    full_path = os.path.join(folder_path, filename)
    return pd.read_excel(full_path)

wse_data_media = read_data(folder_path, 'WSE_data_media_output.xlsx')

wse_data_media["caption_short"] = wse_data_media["caption"].astype(str).str.slice(0, 50)

wse_data_media["engagement_score"] = wse_data_media["like_count"] + wse_data_media["comments_count"]

max_eng = wse_data_media["engagement_score"].max()

wse_data_media["engagement_rate_percent"] = (wse_data_media["engagement_score"] / max_eng) * 100
wse_data_media["engagement_rate_percent"] = wse_data_media["engagement_rate_percent"].round(2)
wse_data_media["engagement_rate_percent"].head()

top10 = wse_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)
fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - WSE", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()

In [12]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# KI_Instagram
def read_data(path, filename):
    full_path = os.path.join(path, filename)
    return pd.read_excel(full_path)

KI_data_media = read_data(path, 'KI_instagram_data_media_output.xlsx')

# 2. Potong caption 50 karakter
# ============================
KI_data_media["caption_short"] = KI_data_media["caption"].astype(str).str.slice(0, 50)
# ============================
KI_data_media["caption_short"] = KI_data_media["caption"].astype(str).str.slice(0, 50)

# ============================
# 3. Hitung Engagement Rate
# Rumus: (likes + comments) / impressions * 100
# Jika impressions tidak ada → pakai reach
# ============================
# Hitung Engagement Rate berdasarkan impressions
KI_data_media["engagement_score"] = KI_data_media["like_count"] + KI_data_media["comments_count"]

max_eng = KI_data_media["engagement_score"].max()

KI_data_media["engagement_rate_percent"] = (KI_data_media["engagement_score"] / max_eng) * 100
KI_data_media["engagement_rate_percent"] = KI_data_media["engagement_rate_percent"].round(2)
KI_data_media["engagement_rate_percent"].head()
top10 = KI_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)

fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - KI", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()

In [13]:
def read_data(path, filename):
    full_path = os.path.join(path, filename)
    return pd.read_excel(full_path)

KI_data_media = read_data(path, 'KI_instagram_data_media_output.xlsx')

# 2. Potong caption 50 karakter
# ============================
KI_data_media["caption_short"] = KI_data_media["caption"].astype(str).str.slice(0, 50)
# ============================
KI_data_media["caption_short"] = KI_data_media["caption"].astype(str).str.slice(0, 50)

# ============================
# 3. Hitung Engagement Rate
# Rumus: (likes + comments) / impressions * 100
# Jika impressions tidak ada → pakai reach
# ============================
# Hitung Engagement Rate berdasarkan impressions
KI_data_media["engagement_score"] = KI_data_media["like_count"] + KI_data_media["comments_count"]

max_eng = KI_data_media["engagement_score"].max()

KI_data_media["engagement_rate_percent"] = (KI_data_media["engagement_score"] / max_eng) * 100
KI_data_media["engagement_rate_percent"] = KI_data_media["engagement_rate_percent"].round(2)
KI_data_media.head()

,id,caption,media_type,media_product_type,media_url,thumbnail_url,permalink,timestamp,like_count,comments_count,caption_counts,date,month_posted,hour,hashtag_counts,caption_short,engagement_score,engagement_rate_percent
0,18097176739749416,Bener-bener couple goals nih 😍 \n_______\n\nLi...,CAROUSEL_ALBUM,FEED,https://scontent-ord5-2.cdninstagram.com/v/t51...,NaN,https://www.instagram.com/p/DQGdtjGkvcf/,2025-10-22T06:12:04+0000,223,1,486,2025-10-22 13:12:04,October,6,7,Bener-bener couple goals nih 😍 \n_______\n\nLi...,224,1.81
1,17890926267359686,Komen yang mau jadi muridnya Mbak Adele 🤚 \n__...,VIDEO,REELS,https://scontent-ord5-2.cdninstagram.com/o1/v/...,https://scontent-ord5-1.cdninstagram.com/v/t51...,https://www.instagram.com/reel/DQGEowLEitf/,2025-10-22T02:35:21+0000,208,10,497,2025-10-22 09:35:21,October,2,7,Komen yang mau jadi muridnya Mbak Adele 🤚 \n__...,218,1.76
2,18093483835834410,Video ini buat kamu yang sering ngerasa stuck ...,VIDEO,REELS,https://scontent-ord5-1.cdninstagram.com/o1/v/...,https://scontent-ord5-1.cdninstagram.com/v/t51...,https://www.instagram.com/reel/DQEj2Pmkp7g/,2025-10-21T12:30:43+0000,53,0,529,2025-10-21 19:30:43,October,12,7,Video ini buat kamu yang sering ngerasa stuck ...,53,0.43
3,18311853946217235,Lagu bunda maia kalo dibawain mbak selena jadi...,CAROUSEL_ALBUM,FEED,https://scontent-ord5-3.cdninstagram.com/v/t51...,NaN,https://www.instagram.com/p/DQD2AJwEvV6/,2025-10-21T05:46:36+0000,377,22,510,2025-10-21 12:46:36,October,5,7,Lagu bunda maia kalo dibawain mbak selena jadi...,399,3.23
4,18109828063582773,Coba absen yang baru tau juga 😌\n_______\n\nLi...,VIDEO,REELS,https://scontent-ord5-3.cdninstagram.com/o1/v/...,https://scontent-ord5-1.cdninstagram.com/v/t51...,https://www.instagram.com/reel/DQDcdR_EtYr/,2025-10-21T02:04:10+0000,1149,33,486,2025-10-21 09:04:10,October,2,7,Coba absen yang baru tau juga 😌\n_______\n\nLi...,1182,9.57


In [14]:
def read_data(path, filename):
    full_path = os.path.join(path, filename)
    return pd.read_excel(full_path)
lister_data_media = read_data(path, 'lister_data_media_output.xlsx')

# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.like_count', 'like_count')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.comments_count', 'comments_count')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.media_url', 'media_url')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.id', 'media_id')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.media_type', 'media_type')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.media_product_type', 'media_product_type')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.caption', 'caption')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.timestamp', 'timestamp')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.permalink', 'permalink')
# lister_data_media.columns = lister_data_media.columns.str.replace('Column1.thumbnail_url', 'thumbnail_url')
# lowercase_columns = lister_data_media.columns.str.lower()
# lister_data_media.columns = lowercase_columns


In [15]:
lister_data_media['timestamp'] = pd.to_datetime(lister_data_media['timestamp'])
lister_data_media['date'] = lister_data_media['timestamp'].dt.date


In [16]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# Lister



# ============================
# 2. Potong caption 50 karakter
# ============================
lister_data_media["caption_short"] = lister_data_media["caption"].astype(str).str.slice(0, 50)
# ============================
lister_data_media["caption_short"] = lister_data_media["caption"].astype(str).str.slice(0, 50)

# ============================
# 3. Hitung Engagement Rate
# Rumus: (likes + comments) / impressions * 100
# Jika impressions tidak ada → pakai reach
# ============================
# Hitung Engagement Rate berdasarkan impressions
lister_data_media["engagement_score"] = lister_data_media["like_count"] + lister_data_media["comments_count"]

max_eng = lister_data_media["engagement_score"].max()

lister_data_media["engagement_rate_percent"] = (lister_data_media["engagement_score"] / max_eng) * 100
lister_data_media["engagement_rate_percent"] = lister_data_media["engagement_rate_percent"].round(2)
lister_data_media.head()

# =============================
# 4. Pilih kolom untuk Top Posts
top10 = lister_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)
# =============================
fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - Lister", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()



In [17]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# The Key_Instagram
def read_data(path, filename):
    full_path = os.path.join(path, filename)
    return pd.read_excel(full_path)


the_key_data_media = read_data(path, 'the_key_data_media_output.xlsx')


the_key_data_media["caption_short"] = the_key_data_media["caption"].astype(str).str.slice(0, 50)

the_key_data_media["caption_short"] = the_key_data_media["caption"].astype(str).str.slice(0, 50)
the_key_data_media["engagement_score"] = the_key_data_media["like_count"] + the_key_data_media["comments_count"]

max_eng = the_key_data_media["engagement_score"].max()

the_key_data_media["engagement_rate_percent"] = (the_key_data_media["engagement_score"] / max_eng) * 100
the_key_data_media["engagement_rate_percent"] = the_key_data_media["engagement_rate_percent"].round(2)
#the_key_data_media.head(10)
top10 = the_key_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)

fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - The Key", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()

In [18]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# KI_Instagram
def read_data(path, filename):
    full_path = os.path.join(path, filename)
    return pd.read_excel(full_path)

titik_nol_data_media = read_data(path, 'titik_nol_data_media_output.xlsx')


titik_nol_data_media["caption_short"] = titik_nol_data_media["caption"].astype(str).str.slice(0, 50)

titik_nol_data_media["engagement_score"] = titik_nol_data_media["like_count"] + titik_nol_data_media["comments_count"]

max_eng = titik_nol_data_media["engagement_score"].max()

titik_nol_data_media["engagement_rate_percent"] = (titik_nol_data_media["engagement_score"] / max_eng) * 100
titik_nol_data_media["engagement_rate_percent"] = titik_nol_data_media["engagement_rate_percent"].round(2)
titik_nol_data_media.head(10)
# =============================
top10 = titik_nol_data_media.sort_values(by="engagement_rate_percent", ascending=False).head(10)

fig = px.bar(top10, x="caption_short", y="engagement_rate_percent", title="Top 10 Posts by Engagement Rate - Titik Nol", template='plotly_dark', color='engagement_rate_percent', 
             labels={'caption_short': 'Caption (shortened)', 'engagement_rate_percent': 'Engagement Rate (%)'})
fig.show()

In [19]:
titik_nol_data_media.columns

Index(['id', 'caption', 'media_type', 'media_product_type', 'media_url',
       'thumbnail_url', 'permalink', 'timestamp', 'like_count',
       'comments_count', 'caption_counts', 'date', 'month_posted', 'hour',
       'hashtag_counts', 'caption_short', 'engagement_score',
       'engagement_rate_percent'],
      dtype='object')

## Engagement Rate by Followers (dilihat dari total followers)

In [20]:
# Asumsi: Tentukan jumlah followers di sini (Ganti dengan angka follower Anda yang sebenarnya)
total_followers = 5000  # Contoh: 100.000 followers

# 1. Pastikan kolom engagement_score sudah dihitung
# lister_data_media["engagement_score"] = lister_data_media["like_count"] + lister_data_media["comments_count"]

# 2. Hitung Engagement Rate by Follower (ERF)
# Rumus: (Engagement Score / Total Followers) * 100
the_key_data_media["engagement_rate_follower"] = (
    the_key_data_media["engagement_score"] / total_followers
) * 100

# 3. Bulatkan hasilnya menjadi dua angka di belakang koma (persentase)
the_key_data_media["engagement_rate_follower"] = the_key_data_media["engagement_rate_follower"].round(2)

# Tampilkan beberapa baris pertama untuk verifikasi
print(the_key_data_media[["like_count", "comments_count", "engagement_score", "engagement_rate_follower"]].head())

   like_count  comments_count  engagement_score  engagement_rate_follower
0           1               0                 1                      0.02
1           1               0                 1                      0.02
2          10               0                10                      0.20
3           4               0                 4                      0.08
4           3               0                 3                      0.06


# Time-Based Analysis
## 1. Posting Frequency Over Time


In [21]:
df_all['account'].unique()


array(['BritishCouncil', 'WallStreetEnglish', 'EF', 'Lister', 'Titik_Nol',
       'KampungInggrisLC', 'The_Key'], dtype=object)

In [22]:
def plot_weekly_posting_october(df_all, accounts):
    """
    Membuat line chart frekuensi posting mingguan
    HANYA untuk bulan Oktober menggunakan kolom 'month_posted'.
    """

    # 1. Filter akun
    df_multi = df_all[df_all['account'].isin(accounts)].copy()

    # Hapus kolom duplikat (sering menyebabkan error "cannot assemble with duplicate keys")
    df_multi = df_multi.loc[:, ~df_multi.columns.duplicated()]

    # =================================================================
    # [BAGIAN BARU] FILTER HANYA BULAN OKTOBER
    # Menggunakan kolom 'month_posted' sesuai permintaan
    # =================================================================
    # Pastikan kolom month_posted ada, jika tidak ada return empty plot / pesan
    if 'month_posted' not in df_multi.columns:
        print("Kolom 'month_posted' tidak ditemukan. Tidak ada data untuk difilter.")
        return

    df_multi = df_multi[df_multi['month_posted'] == 'October']

    # 2. Pastikan kolom tanggal benar 
    # (Kita tetap butuh 'date' dikonversi untuk sumbu X grafik dan pengelompokan mingguan)
    if 'date' not in df_multi.columns:
        print("Kolom 'date' tidak ditemukan. Pastikan kolom tanggal tersedia sebelum plotting.")
        return

    df_multi['date'] = pd.to_datetime(df_multi['date'], errors='coerce')
    df_multi = df_multi.dropna(subset=['date'])

    # 3. Hitung posting mingguan per akun
    # Menggunakan pd.Grouper(key='date') agar tidak perlu set_index manual
    # Ini juga mengatasi FutureWarning yang muncul sebelumnya
    weekly_posting = (
        df_multi
        .groupby(['account', pd.Grouper(key='date', freq='W')])
        .size()
        .reset_index(name='post_count')
    )

    # 4. Plot
    fig = px.line(
        weekly_posting,
        x="date",
        y="post_count",
        color="account",
        title=f"Weekly Posting Frequency in October: {', '.join(accounts)}",
        labels={
            "date": "Weekly Period",
            "post_count": "Post Count",
            "account": "Account"
        },
        template="plotly_dark"
    )

    # Mengatur sumbu X agar menampilkan detail per minggu (karena datanya cuma 1 bulan)
    fig.update_xaxes(
        dtick="W1", 
        tickformat="%Y-%m-%d"
    )

    fig.show()

In [23]:
# define the accounts list (use all accounts present in df_all)
all_accounts = df_all['account'].unique().tolist()

# call the plotting function
plot_weekly_posting_october(df_all, all_accounts)

In [24]:
EF_data_media.dtypes

id                           int64
caption                     object
media_type                  object
media_product_type          object
media_url                   object
thumbnail_url               object
permalink                   object
timestamp                   object
like_count                   int64
comments_count               int64
caption_counts               int64
date                       float64
month_posted                object
hour                         int64
hashtag_counts               int64
caption_short               object
engagement_score             int64
engagement_rate_percent    float64
dtype: object

## 2. Best Time to Post (Post Hour VS Total Like Counts)

In [25]:
# 1. Pastikan kolom 'account' ada sebagai penanda (Opsional jika sudah ada)

BC_data_media['account'] = 'British Council'
wse_data_media['account'] = 'Wall Street English'
lister_data_media['account'] = 'Lister'


df_combined = pd.concat([BC_data_media, wse_data_media, lister_data_media], ignore_index=True)
df_grouped = (
    df_combined
    .groupby(['hour', 'account'])['like_count']
    .sum()
    .reset_index()
)
df_grouped = df_grouped.sort_values('hour')
top_combined = df_grouped.sort_values(by='like_count', ascending=False).head(6)

fig = px.bar(
    top_combined,
    x='hour',
    y='like_count',
    color='account',  # Supaya tahu postingan juara ini milik siapa
    title='Top 5 Postingan dengan Likes Tertinggi (Gabungan)',
    labels={'hour': 'Jam Posting', 'like_count': 'Jumlah Likes'},
    template='plotly_dark'
)

# Trik agar batang tidak menumpuk jika ada jam yang sama di Top 5
fig.update_layout(xaxis={'type': 'category'})

fig.show()

In [26]:
# Ambil Top 5 dari data GABUNGAN
# 1. Pastikan kolom 'account' ada sebagai penanda (Opsional jika sudah ada)
titik_nol_data_media['account'] = 'Titik Nol'
KI_data_media['account'] = 'Kampung Inggris LC'
EF_data_media['account'] = 'EF'
df_combined = pd.concat([KI_data_media, titik_nol_data_media, EF_data_media], ignore_index=True)
df_grouped = (
    df_combined
    .groupby(['hour', 'account'])['like_count']
    .sum()
    .reset_index()
)
df_grouped = df_grouped.sort_values('hour')
top_5_combined = df_grouped.sort_values(by='like_count', ascending=False).head(10)

fig = px.bar(
    top_5_combined,
    x='hour',
    y='like_count',
    color='account',  # Supaya tahu postingan juara ini milik siapa
    title='Top 5 Postingan dengan Likes Tertinggi (Gabungan)',
    labels={'hour': 'Jam Posting', 'like_count': 'Jumlah Likes'},
    template='plotly_dark'
)

# Trik agar batang tidak menumpuk jika ada jam yang sama di Top 5
fig.update_layout(xaxis={'type': 'category'})

fig.show()


In [27]:
import plotly.express as px

top_1 = the_key_data_media.sort_values(by='like_count', ascending=False).head()

fig = px.bar(
    top_1,
    x='hour',
    y='like_count',
    title='Jam Posting dengan Like Count Tertinggi',
    labels={'hour': 'Jam Posting', 'like_count': 'Total Like Count'},
    template='plotly_dark'
)

fig.show()


In [28]:
%pip install statsmodels

import statsmodels.api as sm

Note: you may need to restart the kernel to use updated packages.


# Content Quality Insights
## 1. Caption Length Analysis 

In [29]:
#pakai df secara manual/satu-satu karena ada perbedaan nama kolom caption_counts khususnya di akun Lister
#jika kolom sesuai bisa langsung pakai df_all

# --- 1. PROSES MANUAL TIAP DATAFRAME ---

# BC
BC_data_media['caption_counts'] = BC_data_media['caption'].astype(str).str.len()
BC_data_media['account'] = 'British Council'

# EF
EF_data_media['caption_counts'] = EF_data_media['caption'].astype(str).str.len()
EF_data_media['account'] = 'EF'

# WSE
wse_data_media['caption_counts'] = wse_data_media['caption'].astype(str).str.len()
wse_data_media['account'] = 'Wall Street English'

# The Key
the_key_data_media['caption_counts'] = the_key_data_media['caption'].astype(str).str.len()
the_key_data_media['account'] = 'The Key'

# Titik Nol
titik_nol_data_media['caption_counts'] = titik_nol_data_media['caption'].astype(str).str.len()
titik_nol_data_media['account'] = 'Titik Nol'

# KI (Kampung Inggris)
KI_data_media['caption_counts'] = KI_data_media['caption'].astype(str).str.len()
KI_data_media['account'] = 'Kampung Inggris LC'

# Lister (PERBAIKAN DI SINI: Samakan nama kolom jadi 'caption_counts')
lister_data_media['caption_counts'] = lister_data_media['caption'].astype(str).str.len()
lister_data_media['account'] = 'Lister'


# --- 2. GABUNGKAN MANUAL (CONCATENATE) ---
list_dataframes = [
    BC_data_media, 
    EF_data_media, 
    wse_data_media, 
    the_key_data_media, 
    titik_nol_data_media, 
    KI_data_media,
    lister_data_media
]

df_gabungan = pd.concat(list_dataframes, ignore_index=True)


# --- 3. VISUALISASI ---
fig = px.scatter(
    df_gabungan,   
    x="caption_counts", 
    y="engagement_rate_percent",
    color="account",  
    title="Caption Length vs. Post Engagement Rate",
    labels={
        "caption_counts": "Caption Length (Characters)",
        "engagement_rate_percent": "Engagement Rate (%)",
        "account": "Account Name"
    },
    #hover_data=["caption"], 
    template="plotly_dark",
    trendline="ols" 
)

fig.show()

# --- 4. ANALISIS STATISTIK ---
# PERBAIKAN: Gunakan Series vs Series. 
# Karena nama kolom sudah diseragamkan, kita cukup panggil 'caption_counts'.
correlation = df_gabungan['caption_counts'].corr(df_gabungan['engagement_rate_percent'])

print(f"Nilai Korelasi Global: {correlation:.4f}")

Nilai Korelasi Global: -0.2132


In [30]:
df_all['account'].unique().tolist()

['BritishCouncil',
 'WallStreetEnglish',
 'EF',
 'Lister',
 'Titik_Nol',
 'KampungInggrisLC',
 'The_Key']

In [31]:
def hashtag_position(text):
    text = str(text)
    if "#" not in text:
        return "no hashtag"

    lines = text.split("\n")
    if any("#" in line for line in lines[-1:]):   # last line
        return "end of caption"
    elif any("#" in line for line in lines[:1]):  # first line
        return "beginning"
    else:
        return "middle"

BC_data_media["hashtag_position"] = BC_data_media["caption"].apply(hashtag_position)
EF_data_media["hashtag_position"] = EF_data_media["caption"].apply(hashtag_position)
wse_data_media["hashtag_position"] = wse_data_media["caption"].apply(hashtag_position)
the_key_data_media["hashtag_position"] = the_key_data_media["caption"].apply(hashtag_position)
titik_nol_data_media["hashtag_position"] = titik_nol_data_media["caption"].apply(hashtag_position)
KI_data_media["hashtag_position"] = KI_data_media["caption"].apply(hashtag_position)
lister_data_media["hashtag_position"] = lister_data_media["caption"].apply(hashtag_position)

#gabungkan 
df_gabungan['hashtag_position'] = df_gabungan['caption'].apply(hashtag_position)

df_gabungan['hashtag_position'].unique()



array(['end of caption', 'no hashtag', 'middle', 'beginning'],
      dtype=object)

In [32]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- Warna konsisten ---
color_map = {
    "no hashtag": "#1f77b4",
    "end of caption": "#ff7f0e",
    "beginning": "#2ca02c",
    "middle": "#9467bd"
}

# --- List akun ---
accounts = df_gabungan["account"].unique()

# --- Subplots (7 donut chart) ---
fig = make_subplots(
    rows=2, cols=4,
    specs=[[{"type": "domain"}]*4, [{"type": "domain"}]*4],
    subplot_titles=accounts
)

row, col = 1, 1

for acc in accounts:
    df_temp = df_gabungan[df_gabungan["account"] == acc]
    counts = df_temp["hashtag_position"].value_counts()

    fig.add_trace(
        go.Pie(
            labels=counts.index,
            values=counts.values,
            hole=0.55,
            name=acc,
            marker=dict(colors=[color_map[l] for l in counts.index])
        ),
        row=row, col=col
    )

    col += 1
    if col > 4:
        col = 1
        row += 1

# --- Layout ---
fig.update_layout(
    height=800,
    width=1300,
    title_text="Hashtag Position Distribution per Account",
    template="plotly_dark",
    showlegend=True
)

fig.show()


In [33]:
# CARA MELIHAT POSISI HASHTAG PER ACCOUNT 
# def hashtag_position(text):
#     text = str(text)
#     if not "#" in text:
#         return "no hashtag"

#     lines = text.split("\n")
#     if any("#" in line for line in lines[-1:]):   # baris terakhir
#         return "end of caption"
#     elif any("#" in line for line in lines[:1]):  # baris pertama
#         return "beginning"
#     else:
#         return "middle"

# lister_data_media["hashtag_position"] = lister_data_media["caption"].apply(hashtag_position)

# lister_data_media["hashtag_position"].value_counts()

# fig = px.pie(
#     names=lister_data_media['hashtag_position'].value_counts().index,
#     values=lister_data_media['hashtag_position'].value_counts().values,
#     hole=0.5,
#     template='plotly_dark',
#     title='Hashtag Position Distribution (Donut Chart)'
# )
# fig.show()



In [34]:
import plotly.express as px

fig = px.violin(
    df_gabungan,
    x="hashtag_position",
    y="engagement_rate_percent",
    color="hashtag_position",
    box=True,              # Menampilkan boxplot di dalam violin
    points="all",          # Menampilkan semua titik data
    hover_data=df_gabungan.columns,
    title="Engagement Rate Distribution based on Hashtag Position",
    labels={
        "hashtag_position": "Hashtag Position",
        "engagement_rate_percent": "Engagement Rate (%)"
    },
    template="plotly_dark"
)

fig.show()


In [35]:
df_gabungan.groupby("hashtag_position")[["like_count", "comments_count"]].mean()



,like_count,comments_count
hashtag_position,,
beginning,59.000000,0.000000
end of caption,1107.705674,21.706714
middle,14.578947,0.263158
no hashtag,61.148936,2.844444


In [36]:
fig = px.density_heatmap(
    df_gabungan,
    x="hashtag_position",
    y="engagement_rate_percent",
    title="Heatmap Panjang Caption vs Engagement Rate",
    labels={
        "hashtag_position": "Posisi Hashtag",
        "engagement_rate_percent": "Engagement Rate (%)"
    },
    template="plotly_dark"
)
fig.show()